In [1]:
from bs4 import BeautifulSoup
import json
import requests
from pathlib import Path
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
api_key  = os.getenv('OPENAI_APIKEY')
subscription_key = os.getenv('BING_APIKEY')
assert subscription_key


#参考文献 https://engineering.mobalab.net/2024/04/25/ より一部抜粋

In [2]:
def check_openai_api_key(api_key):
    client = OpenAI(api_key=api_key)
    try:
        # モデルのリストを取得して、API キーの有効性を確認
        client.models.list()
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return False  # API キーが無効またはエラー
    else:
        return True  # API キーが有効

# 実際の API キーをここに入力
is_valid = check_openai_api_key(api_key)
print(is_valid)

True


In [ ]:
def get_web_content(sentence, num_results=1):

    headers = {
    }
    params = {
        "key": subscription_key,
        "cx": search_engine,
        "q": sentence,                     # 検索クエリ
#        "textDecorations": True,
#        "textFormat": "HTML",
#        "count": num_results               # 検索結果の数を指定
    }
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()  # ステータスコードが200でない場合はエラーを発生させる
    search_results = response.json()  # 検索結果をJSON形式で取得

    # search_resultsからWebページのURLのみを抽出する
    if 'items' in search_results:
        web_pages = search_results['items']
        urls = [page['formattedUrl'] for page in web_pages]

    all_text = ""  # 全てのテキストをこの文字列に追加
    print(urls)
    # 各URLからHTMLを取得し、<body>タグ内のテキストを抽出
    for url in urls:
        response = requests.get(url)
 #       response.raise_for_status()  # ステータスコードが200でない場合にエラーを発生させる

        html_content = response.content.decode('utf-8')

        # HTMLの解析
        soup = BeautifulSoup(html_content, 'html.parser')
        body = soup.find('body')

        if body:
            # <body>内のすべてのテキストを抽出し、HTMLタグを除去
            body_text = body.get_text(separator=' ', strip=True)
            all_text += body_text + "\n\n"  # テキストを全体のテキストに追加
        else:
            print(f"No <body> tag found for URL: {url}")

    if all_text:
        message = f"以下は「{sentence}」に関する情報です。正確な情報を取得してください。\n{all_text}"
        completion = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "user", "content": message}
            ]
        )

        return completion.choices[0].message.content
    else:
        return "No text was extracted from the provided URLs."


In [5]:


# APIクライアントの初期化

client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)
search_url = "https://www.googleapis.com/customsearch/v1"


tools = [
    {
#        "tool_call_id": "get_web_content",
        "type": "function",
        "function": {
            "name": "get_web_content",
            "description": "引数の文章でWeb上を検索して正確な情報を返す関数。",
            "parameters": {
                "type": "object",
                "properties": {
                    "sentence": {
                        "type": "string",
                        "description": "検索する文章。",
                    },
                },
                "required": ["sentence"],
            },
        },
    }
]

# 初期システムメッセージの設定
messages = [
    {"role": "system", "content": "あなたは優秀なアシスタントです。聞かれた情報を学習していない場合はWeb上を検索して正確な情報を返します。"}
]



In [ ]:
while True:
    # キーボードからユーザー入力を受け取る
    user_input = input("\nあなたの質問を入力してください (終了するには 'quit' と入力): ")
    if user_input.lower() == 'quit':
        break

    # ユーザーメッセージを追加
    messages.append({"role": "user", "content": user_input})

    # ChatGPTモデルに問い合わせ
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto" # assistant_response.tool_calls
    )

    assistant_response = completion.choices[0].message
    tool_calls = assistant_response.tool_calls
    print(tool_calls)
    if tool_calls:

        # すべてのtool_callsを処理してリストにまとめる
        tool_calls_list = []
        for tool_call in tool_calls:
            tool_call_id = tool_call.id
            tool_call_type = tool_call.type
            tool_call_function = tool_call.function

            tool_calls_list.append({
                "id": tool_call_id,
                "type": tool_call_type,
                "function": {
                    "name": tool_call_function.name,
                    "arguments": tool_call_function.arguments
                }
            })

        # tool_callsの全情報をmessagesに一度に追加
        messages.append({
            "role": "assistant",
            "tool_calls": tool_calls_list
        })

        # ツールがget_web_contentである場合、その関数を実行する
        for tool_call in tool_calls_list:
            print(tool_call['function']['name'])
            if tool_call['function']['name'] == "get_web_content":
                arguments = json.loads(tool_call['function']['arguments'])
                sentence = arguments['sentence']
                web_content = get_web_content(sentence, num_results=1)
                messages.append({"tool_call_id": tool_call['id'], "role": "tool", "name": tool_call['function']['name'], "content": web_content})

        # すべての変更をAPIに送信して結果を取得
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        assistant_response = completion.choices[0].message.content
        print("Assistant:", assistant_response)
        messages.append({"role": "assistant", "content": assistant_response})
    else:
        assistant_response = assistant_response.content
        print("Assistant:", assistant_response)

        messages.append({"role": "assistant", "content": assistant_response})


None
Assistant: Hello! How can I assist you today?
[ChatCompletionMessageToolCall(id='call_Ccw3rkh3XlJ1Pv6EFH8qXFdX', function=Function(arguments='{"sentence":"東京の現在の天気"}', name='get_web_content'), type='function')]
get_web_content
['https://weather.yahoo.co.jp/weather/jp/13/4410.html', 'https://weathernews.jp/onebox/tenki/tokyo/', 'https://tenki.jp/amedas/3/16/', 'https://www.nhk.or.jp/kishou-saigai/pref/weather/tokyo/', 'https://tenki.jp/radar/3/16/', 'https://www.jma.go.jp/bosai/amedas/', 'https://tokyo-ame.jwa.or.jp/', 'https://www.toshin.com/weather/detail?id=56682', 'https://typhoon.yahoo.co.jp/weather/jp/warn/13/', 'https://www.data.jma.go.jp/stats/data/mdrr/index.html']
Assistant: 申し訳ありませんが、現在の天気情報を直接取得することはできません。東京の現在の天気を知るには、天気予報サイトやアプリを利用することをお勧めします。例えば、Weather.comや気象庁の公式サイトなどをご覧ください。
None
Assistant: Slackの運営会社は「Slack Technologies, Inc.」です。Slackは2021年にSalesforceによって買収され、現在はSalesforceの傘下として運営されています。
